#### Parte I - Análisis de la base de hogares y cálculo de pobreza

##### 1. Analisis exploratorio

Comezamos pensando, solo intuitvamente cuales a priori son las variables que podrian ser utilies para predecir la pobreza y perfeccionar las estimaciones el TP2:

Dentro de la varibles en la encuesta de hogares consideramos relevantes las caracterisicas de la vivienda:
* El tipo de vivienda (IV1)
* Cantidad de habitaciones (IV2)
* Material del techo (V4)
* Revestimiento del techo (IV5)
* Tipo de conexion a agua (IV6 y IV7)
* Tener baño (IV8), tipo de baño (IV9) y equipamiento (IV10)
* Tipo de desague (IV11)
* Zona de ubicacion: cercano a basural (IV12_1), zona inundable (IV12_2) y/o villa de emergecia (IV12_3)

Las caractersiticas habitacionales:
* Cantidad de ambientes (II1)
* Cantidad de dormitorios (II2)
* Habitaccion para trabajo (II3), exlusivamente (II6)
* Tienen: cuarto de cocina (II4_1), lavadero (II4_2), garage (II4_3), cuantos de estos se usan para dormir (II5).
* Regimen de tenencia (II7)
* Combustible para cocinar (II8)
* Propiedad del baño (II9)

Variables de tipo de ingresos: V1 a 18. Y variables de trabajo infantil: V19_A y V19_B

Integrantes del hogar: total (IX_Tot) y menores de 10 (IX_men10).

No icluimos las que son mediciones de ingresos al igual que en el TP2.





2. Continuamos descargando la base de hogares y cargandola

In [1]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Definimos el directorio
os.chdir('C:/Users/Usuario\Desktop/MAESTRIA\Big Data/TPs/BigData/TP3')

In [2]:
# Cargamos los datos, menteniendo solo las observaciones para el Gran Buenis Aires y la Ciudad de Buenos Aires:
hogares = pd.read_excel("inputs/usu_hogar_T123.xlsx")
hogares = hogares[(hogares['AGLOMERADO']==32) | (hogares['AGLOMERADO']==33)]
individual = pd.read_excel("inputs/usu_individual_T123.xlsx")
individual = individual[(individual['AGLOMERADO']==32) | (individual['AGLOMERADO']==33)]

3. Unimos la tablas de Individuos y hogares

In [3]:
# Unimos las tablas
# left join individual con hogares por CODUSU
df = pd.merge(individual, hogares, on=['CODUSU','PONDERA','PONDIH','ITF'], how='left')

4. Herramientas de limpieza de datos

5. Limpieza de Datos

6. Estadisticas descriptivas

7. Calculo de Adulto Equivalente y Requerimentos del Hogar

In [4]:
# Cargaos la tabla de equivalencia de adultos
adulto_equiv_data = pd.read_excel("inputs/tabla_adulto_equiv.xlsx")

# melt adulto_equiv_data Muejeres y Hombres
adulto_equiv_data = pd.melt(adulto_equiv_data, id_vars=['Edad'], value_name= 'adulto_equiv')

# Change values of variable column to 1 if Hom and 2 if Muj
adulto_equiv_data['variable'] = np.where(adulto_equiv_data['variable']=='Mujeres', 2, 1)
# Rename variable column to Sexo
adulto_equiv_data = adulto_equiv_data.rename(columns={'variable': 'Sexo'})

#Función que lee los valores de edad en números CH06 y me lo impacta en la categoría etarea correspondiende de la nueva tabla.
def rango_edad(edad):
    if edad < 18:
        rangoetareo = str(edad)+" años"
    elif 17 < edad and edad < 30:
        rangoetareo = "18 a 29 años"
    elif 29 < edad and edad < 46:
        rangoetareo = "30 a 45 años"
    elif 45 < edad and edad < 61:
        rangoetareo = "46 a 60 años"
    elif 60 < edad and edad < 76:
        rangoetareo = "61 a 75 años"
    elif edad > 75:
        rangoetareo = "más de 75 años"
 #   elif for n in range(1,18):
  #      rangoetareo = 
        
    else:
        rangoetareo = "ERROR"
    return rangoetareo

#Aplico la función a mi tabla para crear la columna deseada
df['rango_etareo'] = df['CH06'].apply(rango_edad)


#Renombro la columna "edad", igual que la de la otra tabla
df = df.rename(columns={'rango_etareo': 'Edad'})
df = df.rename(columns={'CH04': 'Sexo'})


# Hacemos el merge de las dos tablas
df = df.merge(adulto_equiv_data, on=['Sexo','Edad'], how='left')
#Sumo para las personas de un mismo hogary loguardo como ad_equiv_hogar
df['ad_equiv_hogar'] = df.groupby('CODUSU')['adulto_equiv'].transform('sum')



8. Particiones segun respuesta y ingreso necesario

In [5]:
# Punto 1.3: Particionamos la muestra segun respondieron o no sobre los ingresos:
respondieron = df[df['ITF'] > 0]
respondieron = respondieron.reset_index(drop=True)
norespondieron = df[df['ITF'] <= 0]
norespondieron = norespondieron.reset_index(drop=True)


In [6]:

# Punto 1.4: Agregar columna que indica el ingreso necesario del hogar para no ser pobre
respondieron['ingreso_necesario'] = respondieron['ad_equiv_hogar'] * 53371.05


9. Generamos una indicadora de persona pobre

In [7]:

# Punto 1.4: Columna que indica si una persona es pobre según el ingreso de su hogar
respondieron.loc[:, 'pobre'] = (respondieron['ingreso_necesario'] > respondieron['ITF']).astype(int)
mean_pobres = respondieron['pobre'].mean()
print(mean_pobres)


0.34741338395823446


10. Calculo de pobreza por hogares (muestra expandida)

In [10]:
# Calaculamos la tasa de pobreza de hogares, expondiendo la muestra con el ponderador de hogares, PONDIH.
# Columna indicadora de pobreza de hogares
respondieron.loc[:, 'pobre_hogar'] = (respondieron['ingreso_necesario'] > respondieron['ITF']).astype(int)


In [13]:
# Ponderar los hogares pobres
respondieron['pobre_hogar_expandido'] = respondieron['PONDIH'] * respondieron['pobre_hogar']
respondieron['pobre_hogar_expandido'] = respondieron['pobre_hogar_expandido'] / respondieron['pobre_hogar'].sum()

# Tasa de pobreza de hogares
mean_pobres_hogar = respondieron['pobre_hogar_expandido'].mean()
print(mean_pobres_hogar)

0.9453900412624066
